In [1]:
# importing the necessary libraries
import altair as alt
import pandas as pd

In [2]:
# reading the csv file we created, and observing it by viewing the head and tail of the data
# we also get some additional info using describe()
data = pd.read_csv("steps.csv")
print("Head: ", data.head())
print("Tail: ", data.tail())
print("Other Info: ", data.describe())

Head:          Date  Amount of Steps  Temperature Weather
0  12/1/2023            14430           54  Cloudy
1  12/2/2023            12227           55  Cloudy
2  12/3/2023             5120           45    Rain
3  12/4/2023            16314           52  Cloudy
4  12/5/2023            10674           37  Cloudy
Tail:            Date  Amount of Steps  Temperature Weather
26  12/27/2023             9280           50    Rain
27  12/28/2023             4802           46    Rain
28  12/29/2023             8082           45    Rain
29  12/30/2023            12117           45    Rain
30  12/31/2023             6859           41   Sunny
Other Info:         Amount of Steps  Temperature
count        31.000000    31.000000
mean       9550.000000    45.870968
std        4424.531418     6.883516
min        1298.000000    34.000000
25%        6191.500000    41.000000
50%       10108.000000    46.000000
75%       12577.000000    50.000000
max       17204.000000    59.000000


In [3]:
# creating a new column in our data called 'Temperature Groups'
data['Temperature Groups'] = ''

"""We assign a temperature group for each row in our data"""
def assign_temperatures():
    # for loop used to iterate through each row in our data
    for i in range(len(data)):
        temperature = data.at[i, 'Temperature']
        # if statements used to identify which temperature group a temperature
        # belongs to
        if 30 <= temperature < 40:
            data.at[i, 'Temperature Groups'] = "30-40"
        elif 40 <= temperature < 50:
            data.at[i, 'Temperature Groups'] = "40-50"
        elif 50 <= temperature < 60:
            data.at[i, 'Temperature Groups'] = "50-60"

# calling our function            
assign_temperatures()

# printing the data to ensure the function works properly
print(data)


          Date  Amount of Steps  Temperature Weather Temperature Groups
0    12/1/2023            14430           54  Cloudy              50-60
1    12/2/2023            12227           55  Cloudy              50-60
2    12/3/2023             5120           45    Rain              40-50
3    12/4/2023            16314           52  Cloudy              50-60
4    12/5/2023            10674           37  Cloudy              30-40
5    12/6/2023             2537           36  Cloudy              30-40
6    12/7/2023             9184           34  Cloudy              30-40
7    12/8/2023            12972           41  Cloudy              40-50
8    12/9/2023            17204           50  Cloudy              50-60
9   12/10/2023             1298           59  Cloudy              50-60
10  12/11/2023            10108           48  Cloudy              40-50
11  12/12/2023            14064           45   Sunny              40-50
12  12/13/2023            16133           45   Sunny            

In [4]:
# creating brush to link our 2 charts together
brush = alt.selection_interval()

# creating a dropdown in order to filter based on the weather type
weather_types = list(data['Weather'].unique())
dropdown_weather = alt.binding_select(options=[None] + weather_types, name='Weather', labels = ['All'] + weather_types)
selection_weather = alt.selection_single(fields=["Weather"], bind=dropdown_weather)


# Creating our scatterplot which displays the amount of steps across the month. Color 
# represents the different temperature groups and our tooltip displays additional information about the day.
dot_dates = alt.Chart(data).mark_point().encode(
    x=alt.X('Date', sort=None, timeUnit='monthdate', title='Date'),
    y=alt.Y('Amount of Steps', title='Amount of Steps'),
    color=alt.condition(brush, "Temperature Groups", alt.value("lightgray")),
    tooltip=[alt.Tooltip('Amount of Steps', title="Amount of Steps"), 
             alt.Tooltip('Date', timeUnit='monthdate', title='Date (2023)'), 
             alt.Tooltip('Weather', title="Weather Type"), alt.Tooltip('Temperature', 
                                                                       title="Temperature (° Fahrenheit)")],   
).add_selection(brush).add_selection(selection_weather
                                    ).transform_filter(selection_weather
                                                      ).properties(width=550, 
                                                                   title='Amount of Steps Throughout December')

# Creating our bar chart which displays the total amount of steps across the days of the week 
# throughout the month. Again, the color represents the different temperature groups and 
# and the tooltip displays additional information. The brush allows for the user to 
# highlight pieces of this chart using the scatterplot
bar_days = alt.Chart(data).mark_bar(cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3).encode(
    x=alt.X('Date', timeUnit='day', title='Day of Week'),
    y=alt.Y('Amount of Steps', title='Total Amount of Steps'),
    color=alt.condition(brush, "Temperature Groups", alt.value("lightgray")),
     tooltip=[alt.Tooltip('Amount of Steps', title="Amount of Steps"), 
              alt.Tooltip('Date', timeUnit='monthdate', title='Date (2023)'), 
              alt.Tooltip('Weather', title="Weather Type"), alt.Tooltip('Temperature', 
                                                                        title="Temperature (° Fahrenheit)")],
).add_selection(selection_weather).transform_filter(selection_weather
                                                   ).properties(width=150, 
                                                                title='Total Steps Across Days of the Week')

# Adding a mean line for the scatterplot
rule = alt.Chart(data).mark_rule(color='red').encode(
    y='mean(Amount of Steps)'
)
# Displaying the visualization
viz = ((dot_dates + rule) | bar_days)
viz

/Users/juliangetsey/opt/anaconda3/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/juliangetsey/opt/anaconda3/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)

In [5]:
viz.save("stepsviz.html")
